In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import time

import pandas as pd
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()

In [2]:
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

### Scraping each result page

In [4]:
# URL of page to be scraped
url = 'https://www.indeed.com/jobs?q=data%20analyst&l=austin'
browser.visit(url)

In [5]:
title_list=[]
company_list=[]
location_list=[]
link_description_list=[]


for x in range(1, 75):
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    results = soup.find_all('div', class_="jobsearch-SerpJobCard")
    
    
    for result in results:
    
        title=result.a['title']
        title_list.append(title)
        company=result.find('span', class_='company').text
        company_list.append(company)
        location=result.find(class_='location').text
        location_list.append(location)
        link_description=result.a['href']
        link_description_list.append(link_description)
        
        #print('page:', x, '-------------')
        #print(title)
        #print(company)
        #print(location)
        #print(link_description)
        #print("-------------------------")

#### Create Pandas DataFrame

In [6]:
job_postings_df = pd.DataFrame({
    "Job Title": title_list,
    "Company": company_list,
    "Location": location_list,
    "Job URL": link_description_list
})

In [7]:
job_postings_df.head()

,Job Title,Company,Location,Job URL
0,Healthcare Data Analyst,\n\n TML MultiState IEBP,"Austin, TX 78754",/pagead/clk?mo=r&ad=-6NYlbfkN0D_AngAViu68QezII...
1,Senior Data Analyst,\n\n ShipStation,"Austin, TX 78756 (Rosedale area)",/pagead/clk?mo=r&ad=-6NYlbfkN0CIRWCei1x3jy3uWA...
2,Search Quality Risk and Fraud Analyst,\n\n Indeed,"Austin, TX 78731",/pagead/clk?mo=r&ad=-6NYlbfkN0CiRNM7CVr8YueLFK...
3,Business Intelligence Analyst,\n\n Indeed,"Austin, TX 78731",/pagead/clk?mo=r&ad=-6NYlbfkN0CiRNM7CVr8YueLFK...
4,TRACS Data Analyst 06-19,\n\n Housing Authority of The City of A...,"Austin, TX 78704 (South Lamar-South Congress a...",/rc/clk?jk=c54e5625ad2987c2&fccid=314136e4f82b...


In [8]:
# Connect to local database
rds_connection_string = "root:KB@127.0.0.1/jobs_db"
engine = create_engine(f'mysql://{rds_connection_string}')

In [9]:
# Use pandas to load DataFrame into database
job_postings_df.to_sql(name='job_postings', con=engine, if_exists='append', index=False)

In [10]:
# query from sql
pd.read_sql_query('select * from job_postings', con=engine).head()

,Job Title,Company,Location,Job URL
0,Healthcare Data Analyst,\n\n TML MultiState IEBP,"Austin, TX 78754",/pagead/clk?mo=r&ad=-6NYlbfkN0D_AngAViu68QezII...
1,Senior Data Analyst,\n\n ShipStation,"Austin, TX 78756 (Rosedale area)",/pagead/clk?mo=r&ad=-6NYlbfkN0CIRWCei1x3jy3uWA...
2,Search Quality Risk and Fraud Analyst,\n\n Indeed,"Austin, TX 78731",/pagead/clk?mo=r&ad=-6NYlbfkN0CiRNM7CVr8YueLFK...
3,Business Intelligence Analyst,\n\n Indeed,"Austin, TX 78731",/pagead/clk?mo=r&ad=-6NYlbfkN0CiRNM7CVr8YueLFK...
4,TRACS Data Analyst 06-19,\n\n Housing Authority of The City of A...,"Austin, TX 78704 (South Lamar-South Congress a...",/rc/clk?jk=c54e5625ad2987c2&fccid=314136e4f82b...


### Scraping the description

In [13]:
description_url_list = ['http://indeed.com' + url for url in link_description_list]

In [ ]:
description_text_list=[]
URL=[]
for urls in description_url_list:
    URL.append(urls)
    response = requests.get(urls)
    soup = BeautifulSoup(response.text, 'html.parser')
    description_results=soup.find_all('div', class_="jobsearch-JobComponent-description")
    for description in description_results:
        actual_description=description.text
    description_text_list.append(actual_description)
#         print(urls)
#         print(actual_description)
#         print('------------------------')
        
       